In [2]:
from smolagents import CodeAgent, LiteLLMModel

In [3]:
model = LiteLLMModel(
    model_id="ollama_chat/qwen2.5-coder:3b",  # This model is a bit weak for agentic behaviours though
    api_base="http://localhost:11434",  # replace with 127.0.0.1:11434 or remote open-ai compatible server if necessary
    api_key="ollama",  # replace with API key if necessary
    num_ctx=8192,  # ollama default is 2048 which will fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
)

agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

c:\Users\Sunil\Learning\ollama_fcc\venv\Lib\site-packages\pydantic\_internal\_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the 118th number in the Fibonacci sequence?                                                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:3b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  a, b = 0, 1                                                                                                      
  for i in range(2, 119):                                                                                          
      a, b = b, a + b                                                                                              
  print(a)                                                                                                         
  final_answer(a)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
1264937032042997393488322

Out - Final answer: 1264937032042997393488322

[Step 0: Duration 10.81 seconds| Input tokens: 2,147 | Output tokens: 119]

1264937032042997393488322

In [4]:
import requests
import bs4

In [6]:
agent = CodeAgent(
    tools=[], model=model, additional_authorized_imports=["requests", "bs4"]
)
agent.run(
    "Could you get me the title of the page at url 'https://huggingface.co/blog/smolervlm'? using requests and bs4"
)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you get me the title of the page at url 'https://huggingface.co/blog/smolervlm'? using requests and bs4   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:3b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  import requests                                                                                                  
  from bs4 import BeautifulSoup                                                                                    
                                                                                                                   
  url = "https://huggingface.co/blog/smolervlm"                                                                    
  response = requests.get(url)                                                                                     
  html_content = response.text                                                                                     
  soup = BeautifulSoup(html_content, 'html.parser')                                                                
  title = soup.find('title').get_text()                                                                            
  print("Page title:", title)                                                                                      
  final_answer(title)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Page title: SmolVLM Grows Smaller – Introducing the 256M & 500M Models!

Out - Final answer: SmolVLM Grows Smaller – Introducing the 256M & 500M Models!

[Step 0: Duration 8.40 seconds| Input tokens: 2,042 | Output tokens: 109]

'SmolVLM Grows Smaller – Introducing the 256M & 500M Models!'

In [9]:
from smolagents import DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()
print(search_tool("Who's the current president of Russia?"))


## Search Results

[List of presidents of Russia - Wikipedia](https://en.wikipedia.org/wiki/List_of_presidents_of_Russia)
The office of the president of Russia is the highest authority in the Russian Federation.The holder is the federation's head of state and has formal presidency over the State Council as well as being the commander in chief of the Russian Armed Forces.The office was introduced in 1918 after the February Revolution with the current office emerging after a referendum of 1991. [1]

[Biographies of presidents of Russia - President of Russia](http://www.en.kremlin.ru/structure/president/presidents)
Since May 8, 2008, Vladimir Putin is a Prime Minister of Russia. 2012 On March 4, 2012, he was elected President of Russia and inaugurated on May 7, 2012. 2018 On March 18, 2018, he was re-elected President of Russia. Assumed office on May 7, 2018. 2024 Won Russia's presidential election held on March 15-17, 2024.

[Putin secures 5th term as Russia's president after preordained

In [10]:
from huggingface_hub import list_models

task = "text-classification"

most_downloaded_model = next(
    iter(list_models(filter=task, sort="downloads", direction=-1))
)
print(most_downloaded_model.id)


cross-encoder/ms-marco-MiniLM-L-6-v2


In [18]:
from smolagents import Tool


class ModelDownloadTool(Tool):
    name = "model_download_tool"
    description = "This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub. It returns the name of the checkpoint."
    inputs = {
        "task": {
            "type": "string",
            "description": "The task for which to get the download count.",
        }
    }
    output_type = "string"

    def forward(self, task: str) -> str:
        most_downloaded_model = next(
            iter(list_models(filter=task, sort="downloads", direction=-1))
        )
        return most_downloaded_model.id


In [ ]:
agent = CodeAgent(tools=[model_download_tool], model=model)
agent.run(
    "Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub?"
)


AttributeError: 'function' object has no attribute 'name'

: 